### Year 3 Topology 4 Contingency Analysis

In [1]:
import pandas as pd
import numpy as np
import re

AC contingency calculation was conducted on the hypothetical SAVNW system for 27 different scenarios and hence on 27 different case files corresponding to Topology 4.

The same configuration files are used for the 27 scenarios and are:
1.  Subsystem file savnw.sub<br>
The subsystem file gives the data about the subsystems studied. <br>

![savnw subsystem file](sub.png)

2.  Monitor file savnw.mon<br>
The monitored data file gives the monitored parameters.<br>

![savnw monitor file](mon.png)

3.  Contingency file savnw.con<br>
The contingency file gives the tested contingencies.<br>

![savnw contingency file](con.png)

For each of the 27 studied scenarios, the API DFAX_2 is used to construct 27 different distribution factor data files corresponding to each .sav file, and .sub, .mon, .con configuration files.<br> 
For each of the 27 scenarios, by running the AC contingency calculation function ACCC_WITH_DSP_3, the contingency solution output .acc files are obtained. <br>

Python code to conduct AC contingency calculation is:


For each of the 27 scenarios, using the contingency solution output files, the results are exported as excel files for further analysis. The results exported are ACCC Analysis Summary, Monitored Branch Flows (MVA), Monitored Bus Voltages. <br>

Python code to export AC contingency solution output file as excel is: 


In the following sections the exported data is imported as a python Dataframe to extract non-converged contingencies, branch overloads and bus upper and lower voltage range violations.  

#### Non-converged contingencies for each scenario
Summary sheet in the ACCC file is used to extract all the contingencies tested. The converged contingencies are extracted either from the Bus Voltage sheet or from the Branch Flow sheet. Difference between the two lists gives the desired non-converged contingency. 
Here the branch flow sheet is used to extract the converged contingencies. For each of the 27 scenarios, the scenario, number of tested contingencies, number of converged contingencies, and the contingencies that did not converge are printed below:

In [2]:
list_gens = [0,50,100]
list_lsc = ['lls','rls','hls']
list_gen_hydro = [400,500,600] 
for gen in list_gens:
    for lsc in list_lsc:
        for gen_hy in list_gen_hydro:
            file_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'.xlsx'
            data_flow = pd.read_excel(file_out, sheet_name='Branch Flow')
            data_f_nonan = data_flow.dropna(how='all').reset_index(drop=True)
            data_f_nonan['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
            data_summary = pd.read_excel(file_out, sheet_name='Summary')
            data_summary_nonan = data_summary.dropna(how='all').reset_index(drop=True)
            data_cont = data_summary_nonan.loc[143:151].reset_index(drop=True)
            # extracting all the tested contingencies 
            tested_conts = list(data_cont['ACCC SOLUTION SUMMARY'].unique())
            ans_list = "".join(tested_conts)
            cont_tested = ans_list.split(',')
            cont_test = [s.strip() for s in cont_tested]
            # extracting contingencies for the studied scenario
            scen_cont = data_f_nonan['CONTINGENCY'].unique()
            # blown up scenarios 
            print('For the scenario Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper())
            print(f'{len(scen_cont)} contingencies were converged out of the {len(cont_test)} contingencies tested, the contingencies that are not converged are', ', '.join(list(set(cont_test)-set(scen_cont)))+'.')

For the scenario Solar = 0 MW, Hydro = 400 MW, LLS
55 contingencies were converged out of the 59 contingencies tested, the contingencies that are not converged are BUS 152, BUS 154, BUS 201, BUS 151.
For the scenario Solar = 0 MW, Hydro = 500 MW, LLS
56 contingencies were converged out of the 59 contingencies tested, the contingencies that are not converged are BUS 152, BUS 201, BUS 151.
For the scenario Solar = 0 MW, Hydro = 600 MW, LLS
52 contingencies were converged out of the 59 contingencies tested, the contingencies that are not converged are BUS 201, BUS 205, BUS 151, SING OPN LIN   11 201-204(1), BUS 204, BUS 152, BUS 202.
For the scenario Solar = 0 MW, Hydro = 400 MW, RLS
54 contingencies were converged out of the 59 contingencies tested, the contingencies that are not converged are BUS 201, BUS 151, SING OPN LIN   20 205-206(1), UNIT 206(1), BUS 152.
For the scenario Solar = 0 MW, Hydro = 500 MW, RLS
50 contingencies were converged out of the 59 contingencies tested, the cont

#### Branch flow overload

For the 27 studied scenarios corresponding to Topology 4, the combined branch flow data for all the studied scenarios extracted is as follows:

In [3]:
list_flow = []
list_gens = [0, 50, 100]
list_lsc = ['lls', 'rls','hls']
list_gen_hydro = [400,500,600] 
for gen in list_gens:
    for lsc in list_lsc:
            for gen_hy in list_gen_hydro:
                file_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'.xlsx'
                data_flow = pd.read_excel(file_out, sheet_name='Branch Flow')
                data_f_nonan = data_flow.dropna(how='all').reset_index(drop=True)
                data_f_nonan['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
                list_flow.append(data_f_nonan)
data_f = pd.concat(list_flow).reset_index(drop=True)

The branches reporting loading greater than 100% is filtered based on each of the studied scenario:

In [4]:
bran_overload = data_f[data_f['% FLOW']>100].reset_index(drop=True)
list_scenario = list(bran_overload['Scenario'].unique())
list_number = []
list_branches = []
for scen in list_scenario:
    scen_load = bran_overload[bran_overload['Scenario']==scen]
    no_overload = len(list(scen_load['BRANCH'].unique()))
    list_number.append(no_overload)
    list_ans = list([[int(s) for s in m.split() if s.isdigit()] for m in scen_load['BRANCH'].dropna().unique()])
    ans = ', '.join([(str(s[0])+'-'+str(s[1])+'('+ str(s[2])+')') for s in list_ans])
    list_branches.append(ans)
dict_overlaod = {
                    'Scenario': list_scenario,
                    'Branches Overloaded': list_branches,
                    'Overload Count':list_number
                }
summary_overload = pd.DataFrame(dict_overlaod, columns=['Scenario', 'Branches Overloaded','Overload Count'])

In [5]:
for scenario in list(summary_overload['Scenario'].unique()):
    count = list(summary_overload[summary_overload['Scenario'] == scenario]['Overload Count'])[0]
    load = list(summary_overload[summary_overload['Scenario'] == scenario]['Branches Overloaded'])[0]
    print(f'For the studied scenario {scenario}, the {count} branches overloaded are {load}.')

For the studied scenario Solar = 0 MW, Hydro = 400 MW, LLS, the 5 branches overloaded are 3001-3003(1), 3001-3003(2), 101-151(1), 154-203(1), 154-205(1).
For the studied scenario Solar = 0 MW, Hydro = 500 MW, LLS, the 8 branches overloaded are 3001-3003(1), 3001-3003(2), 101-151(1), 154-203(1), 203-205(1), 203-205(2), 153-3006(1), 3005-3006(1).
For the studied scenario Solar = 0 MW, Hydro = 600 MW, LLS, the 7 branches overloaded are 3001-3003(1), 3001-3003(2), 101-151(1), 154-203(1), 203-205(1), 203-205(2), 153-154(1).
For the studied scenario Solar = 0 MW, Hydro = 400 MW, RLS, the 9 branches overloaded are 3001-3003(1), 3001-3003(2), 3003-3005(1), 3003-3005(2), 101-151(1), 154-203(1), 154-205(1), 203-205(1), 203-205(2).
For the studied scenario Solar = 0 MW, Hydro = 500 MW, RLS, the 8 branches overloaded are 3001-3003(1), 3001-3003(2), 3003-3005(1), 3003-3005(2), 101-151(1), 154-203(1), 154-205(1), 153-154(1).
For the studied scenario Solar = 0 MW, Hydro = 600 MW, RLS, the 11 branches

#### Bus voltage range violations

For the 27 studied scenarios corresponding to Topology 4, the combined bus voltage data for all the studied scenarios extracted is as follows:

In [6]:
list_volt = []
list_gens = [0, 50, 100]
list_lsc = ['lls', 'rls','hls']
list_gen_hydro = [400,500,600] 
for gen in list_gens:
    for lsc in list_lsc:
            for gen_hy in list_gen_hydro:
                file_out = 'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_' + str(lsc)+'.xlsx'
                data_v = pd.read_excel(file_out, sheet_name='Bus Voltage', usecols = ['BUS', 'RECORD', 'TYPE', 'MIN/DROP', 'MAX/RISE', 'CONTINGENCY',
                                        'BASE VOLTS', 'CONT VOLTS', 'DEVIATION', 'RANGE VIO', 'DEV VIO'] )
                data_vo = data_v.dropna(how='all').reset_index(drop=True)
                data_vo['Scenario']= 'Solar = ' + str(gen) + ' MW, ' +'Hydro = ' +str(gen_hy) +' MW, ' + lsc.upper()
                list_volt.append(data_vo)
data_volt = pd.concat(list_volt).reset_index(drop=True)

##### Bus voltage lower range violation (Bus Voltage < 0.9 PU)

The buses reporting lower voltage range violation (bus voltage < 0.9 PU) are filtered based on each of the studied scenario:

In [7]:
data_low = data_volt[data_volt['CONT VOLTS']<0.9].reset_index(drop=True)
list_vvscenario = list(data_low['Scenario'].unique())
list_numberl = []
list_busesl = []
for lscen in list_vvscenario:
    scen_voltl = data_low[data_low['Scenario']==lscen]
    no_downvolt = len(list(scen_voltl['BUS'].unique()))
    list_numberl.append(no_downvolt)
    list_ansl = [s.strip().split(' ')[0] for s in list(scen_voltl['BUS'].unique())]
    ansl = ', '.join(list_ansl)
    list_busesl.append(ansl)
dict_overlaodl = {
                    'Scenario': list_vvscenario,
                    'Buses': list_busesl,
                    'Upper violation count':list_numberl
                }
summary_lowerbus = pd.DataFrame(dict_overlaodl, columns=['Scenario', 'Buses','Upper violation count'])

In [8]:
for lscenario in list(summary_lowerbus['Scenario'].unique()):
    countl = list(summary_lowerbus[summary_lowerbus['Scenario'] == lscenario]['Upper violation count'])[0]
    lowerl = list(summary_lowerbus[summary_lowerbus['Scenario'] == lscenario]['Buses'])[0]
    print(f'For the studied scenario {lscenario}, the {countl} buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses {lowerl}.')

For the studied scenario Solar = 0 MW, Hydro = 400 MW, LLS, the 6 buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses 154, 205, 203, 103, 3008, 206.
For the studied scenario Solar = 0 MW, Hydro = 500 MW, LLS, the 11 buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses 103, 154, 203, 204, 205, 3008, 153, 3005, 3006, 3007, 206.
For the studied scenario Solar = 0 MW, Hydro = 600 MW, LLS, the 4 buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses 154, 205, 3007, 206.
For the studied scenario Solar = 0 MW, Hydro = 400 MW, RLS, the 9 buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses 103, 153, 154, 203, 204, 205, 3007, 3008, 3006.
For the studied scenario Solar = 0 MW, Hydro = 500 MW, RLS, the 2 buses reporting lower voltage range violation (bus voltage < 0.9 PU) are buses 3007, 3008.
For the studied scenario Solar = 0 MW, Hydro = 600 MW, RLS, the 7 buses reporting lower voltage range v

##### Bus voltage upper range violation (Bus Voltage > 1.1 PU)

The buses reporting upper voltage range violation (bus voltage > 1.1 PU) are filtered based on each of the studied scenario:

In [9]:
data_high = data_volt[data_volt['CONT VOLTS']>1.1].reset_index(drop=True)
list_vscenario = list(data_high['Scenario'].unique())
list_number = []
list_buses = []
for scen in list_vscenario:
    scen_volt = data_high[data_high['Scenario']==scen]
    no_upvolt = len(list(scen_volt['BUS'].unique()))
    list_number.append(no_upvolt)
    list_ans = [s.strip().split(' ')[0] for s in list(scen_volt['BUS'].unique())]
    ans = ', '.join(list_ans)
    list_buses.append(ans)
dict_overlaod = {
                    'Scenario': list_vscenario,
                    'Buses': list_buses,
                    'Upper violation count':list_number
                }
summary_upperbus = pd.DataFrame(dict_overlaod, columns=['Scenario', 'Buses','Upper violation count'])

In [10]:
for scenario in list(summary_upperbus['Scenario'].unique()):
    count = list(summary_upperbus[summary_upperbus['Scenario'] == scenario]['Upper violation count'])[0]
    upper = list(summary_upperbus[summary_upperbus['Scenario'] == scenario]['Buses'])[0]
    print(f'For the studied scenario {scenario}, the {count} buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses {upper}.')

For the studied scenario Solar = 0 MW, Hydro = 600 MW, LLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.
For the studied scenario Solar = 0 MW, Hydro = 600 MW, HLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.
For the studied scenario Solar = 50 MW, Hydro = 600 MW, HLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.
For the studied scenario Solar = 100 MW, Hydro = 600 MW, LLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.
For the studied scenario Solar = 100 MW, Hydro = 400 MW, RLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.
For the studied scenario Solar = 100 MW, Hydro = 600 MW, HLS, the 2 buses reporting upper voltage range violation (bus voltage > 1.1 PU) are the buses 211, 212.


#### Conclusion

Year 3 Topology 4 contingency violation analysis was carried out to extract the following: 
1. Branches loaded more than 100% of their rating 
2. Buses violating upper range voltage of 1.1 PU
3. Buses violating lower range voltage of 0.9 PU